In [1]:
import pandas as pd
import itertools
import gene_conversion_tools as gct

## Load InBio_Map Raw Data
#### Source: https://www.intomics.com/inbio/map/#downloads
Downloaded: November 30, 2016  
Last Updated: September 12, 2015   

In [5]:
wd = '/cellar/users/jkhuang/Data/Projects/Network_Analysis/Data/'
InBio_Map_Raw = pd.read_csv(wd+'Network_Data_Raw/InBio_Map_core_2016_09_12/core.psimitab',sep='\t', header=-1)
print 'Raw edge count in InBio_Map:', len(InBio_Map_Raw)

Raw edge count in InBio_Map: 625641


In [8]:
# Extract gene list
InBio_Map_Human_Genes = list(set(InBio_Map_Raw[0]).union(set(InBio_Map_Raw[1])))
InBio_Map_Human_Genes = [str(gene) for gene in InBio_Map_Human_Genes]

## Convert Genes

In [9]:
# Construct list of genes to be submitted to MyGene.Info API
query_string, valid_genes, invalid_genes = gct.query_constructor(InBio_Map_Human_Genes)

17653 Valid Query Genes
0 Invalid Query Genes


In [12]:
# Set scopes (gene naming systems to search)
scopes = "uniprot"

# Set fields (systems from which to return gene names from)
fields = "symbol, entrezgene"

In [13]:
# Query MyGene.Info
match_list = gct.query_batch(query_string, scopes=scopes, fields=fields)
print len(match_list), 'Matched query results'

Batch query complete: 19.89 seconds
17971 Matched query results


In [14]:
match_table_trim, query_to_symbol, query_to_entrez = gct.construct_query_map_table(match_list, valid_genes)

Queries without full matching results found: 408

222 Queries with mutliple matches found

Query mapping table/dictionary construction complete: 79.93 seconds


## Construct Converted Network

In [17]:
query_edgelist = InBio_Map_Raw[[0,1]].values.tolist()
query_edgelist_fmt = [[edge[0].split(':')[1],edge[1].split(':')[1]] for edge in query_edgelist]

In [21]:
InBio_Map_edgelist_symbol = gct.convert_edgelist(query_edgelist_fmt, query_to_symbol)
InBio_Map_edgelist_symbol_filt = gct.filter_converted_edgelist(InBio_Map_edgelist_symbol)
gct.write_edgelist(InBio_Map_edgelist_symbol_filt, wd+'Network_SIFs_Symbol/InBio-Map_Symbol.sif')

625641 input edges
2954 self-edges removed
11517 edges with un-mapped genes removed
4976 duplicate edges removed
Edge list filtered: 0.61 seconds
606194 Edges remaining
Edge list saved: 1.05 seconds


In [22]:
InBio_Map_edgelist_entrez = gct.convert_edgelist(query_edgelist_fmt, query_to_entrez)
InBio_Map_edgelist_entrez_filt = gct.filter_converted_edgelist(InBio_Map_edgelist_entrez)
gct.write_edgelist(InBio_Map_edgelist_entrez_filt, wd+'Network_SIFs_Entrez/InBio-Map_Entrez.sif')

625641 input edges
7536 self-edges removed
11091 edges with un-mapped genes removed
4975 duplicate edges removed
Edge list filtered: 0.52 seconds
602039 Edges remaining
Edge list saved: 0.41 seconds
